## Inicializar la Dimensión Cliente
Importaciones

In [163]:
import pandas as pd
from sqlalchemy import create_engine
import yaml
from datetime import  timedelta

# Añadir conexión a la base de datos

In [164]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Consulta para extraer la información de servicios con sus novedades

In [165]:
df_datamart= pd.read_sql('SELECT * FROM public.trans_novedades', etl_conn)
df_datamart.head()

,key_trans_novedad,novedad_servicio_id,fecha_novedad,tipo_novedad_id,descripcion_novedad,es_prueba_x,mensajero_id,descripcion_servicio,nombre_solicitante,fecha_solicitud,...,multiples_origenes,asignar_mensajero,es_prueba_y,descripcion_cancelado,sede_id,tipo_servicio,origen_servicio_direccion,origen_ciudad_nombre,destino_servicio_direccion,destino_ciudad_nombre
0,0,4,2023-11-30 05:00:00+00:00,1,A,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,False,True,N/A,7,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI
1,1,5,2023-11-30 05:00:00+00:00,1,Halo,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,False,True,N/A,7,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI
2,2,6,2023-11-30 05:00:00+00:00,1,A,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,False,True,N/A,7,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI
3,3,7,2023-11-30 05:00:00+00:00,1,B,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,False,True,N/A,7,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI
4,4,8,2023-11-30 05:00:00+00:00,1,A,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,False,True,N/A,7,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI


# Organizar la tabla de hecho novedades

In [166]:
df_dim_cliente = pd.read_sql('SELECT * FROM dim_cliente', etl_conn)
df_dim_mensajero = pd.read_sql('SELECT * FROM dim_mensajero', etl_conn)
df_dim_sede = pd.read_sql('SELECT * FROM dim_sede', etl_conn)
df_dim_hora = pd.read_sql('SELECT * FROM dim_hora', etl_conn)
df_dim_fecha = pd.read_sql('SELECT * FROM dim_fecha', etl_conn)

df_dim_fecha.head()


,key_dim_fecha,fecha_completa,fecha_id,año,mes,nombre_mes,semana,dia,nombre_dia,trimestre,is_Holiday,holiday,saved,weekend,es_dia_laboral,es_fin_de_semana,es_feriado
0,0,2023-01-01,20230101,2023,1,January,52,1,Sunday,1,True,Año Nuevo,2024-11-04,True,False,True,True
1,1,2023-01-02,20230102,2023,1,January,1,2,Monday,1,False,None,2024-11-04,False,True,False,False
2,2,2023-01-03,20230103,2023,1,January,1,3,Tuesday,1,False,None,2024-11-04,False,True,False,False
3,3,2023-01-04,20230104,2023,1,January,1,4,Wednesday,1,False,None,2024-11-04,False,True,False,False
4,4,2023-01-05,20230105,2023,1,January,1,5,Thursday,1,False,None,2024-11-04,False,True,False,False


# cambiar cliente info por dim cliente

In [160]:
# Merge para obtener key_dim_cliente basado en cliente_id
df_datamart = df_datamart.merge(df_dim_cliente[['cliente_id', 'key_dim_cliente']],
                                on='cliente_id',
                                how='left')
# Eliminar la columna cliente_id original
df_datamart.drop(columns=['cliente_id'], inplace=True)

df_datamart.head()

,key_trans_novedad,novedad_servicio_id,fecha_novedad,tipo_novedad_id,descripcion_novedad,es_prueba_x,mensajero_id,descripcion_servicio,nombre_solicitante,fecha_solicitud,...,asignar_mensajero,es_prueba_y,descripcion_cancelado,sede_id,tipo_servicio,origen_servicio_direccion,origen_ciudad_nombre,destino_servicio_direccion,destino_ciudad_nombre,key_dim_cliente
0,0,4,2023-11-30 05:00:00+00:00,1,A,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,True,N/A,7,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,7
1,1,5,2023-11-30 05:00:00+00:00,1,Halo,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,True,N/A,7,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,7
2,2,6,2023-11-30 05:00:00+00:00,1,A,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,True,N/A,7,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,7
3,3,7,2023-11-30 05:00:00+00:00,1,B,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,True,N/A,7,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,7
4,4,8,2023-11-30 05:00:00+00:00,1,A,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,True,N/A,7,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,7


# cambiar sede info por dim sede

In [167]:
# Merge para obtener key_dim_sede basado en sede_id
df_datamart = df_datamart.merge(df_dim_sede[['sede_id', 'key_dim_sede']],
                                on='sede_id',
                                how='left')
# Eliminar la columna sede_id original
df_datamart.drop(columns=['sede_id'], inplace=True)

df_datamart.head()

,key_trans_novedad,novedad_servicio_id,fecha_novedad,tipo_novedad_id,descripcion_novedad,es_prueba_x,mensajero_id,descripcion_servicio,nombre_solicitante,fecha_solicitud,...,multiples_origenes,asignar_mensajero,es_prueba_y,descripcion_cancelado,tipo_servicio,origen_servicio_direccion,origen_ciudad_nombre,destino_servicio_direccion,destino_ciudad_nombre,key_dim_sede
0,0,4,2023-11-30 05:00:00+00:00,1,A,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,False,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17
1,1,5,2023-11-30 05:00:00+00:00,1,Halo,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,False,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17
2,2,6,2023-11-30 05:00:00+00:00,1,A,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,False,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17
3,3,7,2023-11-30 05:00:00+00:00,1,B,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,False,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17
4,4,8,2023-11-30 05:00:00+00:00,1,A,True,7,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,False,False,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17


# cambiar mensajero info por dim mensajero

In [168]:
# Merge para obtener key_dim_mensajero basado en mensajero_id
df_datamart = df_datamart.merge(df_dim_mensajero[['mensajero_id', 'key_dim_mensajero']],
                                on='mensajero_id',
                                how='left')

# Convertir key_dim_mensajero a entero inmediatamente después del merge
df_datamart['key_dim_mensajero'] = df_datamart['key_dim_mensajero'].astype(int)

# Eliminar la columna mensajero_id original
df_datamart.drop(columns=['mensajero_id'], inplace=True)



In [169]:
df_datamart.head()


,key_trans_novedad,novedad_servicio_id,fecha_novedad,tipo_novedad_id,descripcion_novedad,es_prueba_x,descripcion_servicio,nombre_solicitante,fecha_solicitud,hora_deseada,...,asignar_mensajero,es_prueba_y,descripcion_cancelado,tipo_servicio,origen_servicio_direccion,origen_ciudad_nombre,destino_servicio_direccion,destino_ciudad_nombre,key_dim_sede,key_dim_mensajero
0,0,4,2023-11-30 05:00:00+00:00,1,A,True,Recoger muestras 8:20 am,chat_GPT,2023-11-14,15:00:56,...,False,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37
1,1,5,2023-11-30 05:00:00+00:00,1,Halo,True,Recoger muestras 8:20 am,chat_GPT,2023-11-14,15:00:56,...,False,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37
2,2,6,2023-11-30 05:00:00+00:00,1,A,True,Recoger muestras 8:20 am,chat_GPT,2023-11-14,15:00:56,...,False,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37
3,3,7,2023-11-30 05:00:00+00:00,1,B,True,Recoger muestras 8:20 am,chat_GPT,2023-11-14,15:00:56,...,False,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37
4,4,8,2023-11-30 05:00:00+00:00,1,A,True,Recoger muestras 8:20 am,chat_GPT,2023-11-14,15:00:56,...,False,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37


# cambiar fecha info por dim fecha
## Fecha solicitud y fecha completa

In [170]:
# Convertir fecha_solicitud y fecha_completa a formato datetime64[ns]
df_datamart['fecha_solicitud'] = pd.to_datetime(df_datamart['fecha_solicitud'])
df_dim_fecha['fecha_completa'] = pd.to_datetime(df_dim_fecha['fecha_completa'])

# Realizar el merge para obtener key_dim_fecha correspondiente a cada fecha_solicitud
df_datamart = df_datamart.merge(df_dim_fecha[['fecha_completa', 'key_dim_fecha']],
                                left_on='fecha_solicitud',
                                right_on='fecha_completa',
                                how='left')

# Renombrar la columna key_dim_fecha a key_dim_fecha_solicitud y eliminar la columna fecha_completa
df_datamart.rename(columns={'key_dim_fecha': 'key_dim_fecha_solicitud'}, inplace=True)
df_datamart.drop(columns=['fecha_completa', 'fecha_solicitud'], inplace=True)

df_datamart.head()

,key_trans_novedad,novedad_servicio_id,fecha_novedad,tipo_novedad_id,descripcion_novedad,es_prueba_x,descripcion_servicio,nombre_solicitante,hora_deseada,nombre_recibe,...,es_prueba_y,descripcion_cancelado,tipo_servicio,origen_servicio_direccion,origen_ciudad_nombre,destino_servicio_direccion,destino_ciudad_nombre,key_dim_sede,key_dim_mensajero,key_dim_fecha_solicitud
0,0,4,2023-11-30 05:00:00+00:00,1,A,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37,317
1,1,5,2023-11-30 05:00:00+00:00,1,Halo,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37,317
2,2,6,2023-11-30 05:00:00+00:00,1,A,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37,317
3,3,7,2023-11-30 05:00:00+00:00,1,B,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37,317
4,4,8,2023-11-30 05:00:00+00:00,1,A,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,True,N/A,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37,317


# fecha novedad

In [178]:
# Convertir 'fecha_novedad' y 'fecha_completa' al formato datetime sin zona horaria
df_datamart['fecha_novedad'] = pd.to_datetime(df_datamart['fecha_novedad']).dt.normalize()  # Solo fecha sin zona horaria
df_dim_fecha['fecha_completa'] = pd.to_datetime(df_dim_fecha['fecha_completa']).dt.normalize()

# Extraer solo la hora en una nueva columna
df_datamart['hora_novedad'] = pd.to_datetime(df_datamart['fecha_novedad']).dt.time

# Realizar el merge para obtener key_dim_fecha correspondiente a cada fecha_novedad
df_datamart = df_datamart.merge(df_dim_fecha[['fecha_completa', 'key_dim_fecha']],
                                left_on='fecha_novedad',
                                right_on='fecha_completa',
                                how='left')

# Renombrar la columna key_dim_fecha a key_dim_fecha_novedad y eliminar la columna fecha_completa
df_datamart.rename(columns={'key_dim_fecha': 'key_dim_fecha_novedad'}, inplace=True)
df_datamart.drop(columns=['fecha_completa'], inplace=True)

# Verificar el resultado
df_datamart.head()


,key_trans_novedad,novedad_servicio_id,fecha_novedad,tipo_novedad_id,descripcion_novedad,es_prueba_x,descripcion_servicio,nombre_solicitante,hora_deseada,nombre_recibe,...,tipo_servicio,origen_servicio_direccion,origen_ciudad_nombre,destino_servicio_direccion,destino_ciudad_nombre,key_dim_sede,key_dim_mensajero,key_dim_fecha_solicitud,hora_novedad,key_dim_fecha_novedad
0,0,4,2023-11-30,1,A,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37,317,00:00:00,333
1,1,5,2023-11-30,1,Halo,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37,317,00:00:00,333
2,2,6,2023-11-30,1,A,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37,317,00:00:00,333
3,3,7,2023-11-30,1,B,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37,317,00:00:00,333
4,4,8,2023-11-30,1,A,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,Clínico,Los angeles distrito Latino,CALI,Los angeles distrito Latino,CALI,17,37,317,00:00:00,333


## transformar hora

In [181]:
df_dim_hora.head()

,key_dim_hora,hora_completa,hora_id,hora,minuto,descripcion
0,0,00:00:00,0,00,00,00:00
1,1,00:01:00,1,00,01,00:01
2,2,00:02:00,2,00,02,00:02
3,3,00:03:00,3,00,03,00:03
4,4,00:04:00,4,00,04,00:04


In [184]:
# Convertir `hora_novedad` en `df_datamart` al formato HH:MM:SS
df_datamart['hora_novedad'] = pd.to_datetime(df_datamart['hora_novedad'], errors='coerce').dt.strftime('%H:%M:%S')

# Convertir `hora_completa` en `df_dim_hora` al formato HH:MM:SS
df_dim_hora['hora_completa'] = pd.to_datetime(df_dim_hora['hora_completa'], errors='coerce').dt.strftime('%H:%M:%S')

# Hacer el cruce para reemplazar `hora_novedad` con `key_dim_hora` correspondiente
# Suponiendo que `key_dim_hora` es el nombre de la columna de clave en `df_dim_hora`
df_datamart = df_datamart.merge(df_dim_hora[['hora_completa', 'key_dim_hora']],
                                how='left', left_on='hora_novedad', right_on='hora_completa')

C:\Users\nathy\AppData\Local\Temp\ipykernel_31144\1229390255.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_dim_hora['hora_completa'] = pd.to_datetime(df_dim_hora['hora_completa'], errors='coerce').dt.strftime('%H:%M:%S')


In [185]:
df_datamart.head(
    
)

,key_trans_novedad,novedad_servicio_id,fecha_novedad,tipo_novedad_id,descripcion_novedad,es_prueba_x,descripcion_servicio,nombre_solicitante,hora_deseada,nombre_recibe,...,destino_servicio_direccion,destino_ciudad_nombre,key_dim_sede,key_dim_mensajero,key_dim_fecha_solicitud,hora_novedad,key_dim_fecha_novedad,key_dim_fecha_novedad,hora_completa,key_dim_hora
0,0,4,2023-11-30,1,A,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,Los angeles distrito Latino,CALI,17,37,317,NaN,333,333,NaN,NaN
1,1,5,2023-11-30,1,Halo,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,Los angeles distrito Latino,CALI,17,37,317,NaN,333,333,NaN,NaN
2,2,6,2023-11-30,1,A,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,Los angeles distrito Latino,CALI,17,37,317,NaN,333,333,NaN,NaN
3,3,7,2023-11-30,1,B,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,Los angeles distrito Latino,CALI,17,37,317,NaN,333,333,NaN,NaN
4,4,8,2023-11-30,1,A,True,Recoger muestras 8:20 am,chat_GPT,15:00:56,Gemini,...,Los angeles distrito Latino,CALI,17,37,317,NaN,333,333,NaN,NaN
